In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
data = pd.read_csv(r'E:\Info_Srping\FWI_Prediction\Data\processed\train_data.csv')
data

,report_date,temperature,dew_point_temperature,relative_humidity,wind_speed,wind_direction,atmospheric_pressure,visibility,precipitation,rain_days,...,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,daily_severity_rating,country,latitude,longitude,elevation
0,2019-08-27 12:00:00,12.0,7.6,74.4,12.0,0,1015.13,30.0,2.6,0,...,6.9,51.1,10.3,0.9,0.6,0.0,United Kingdom,54.502224,-2.68500,252.0
1,2018-06-06 12:00:00,11.1,1.7,52.0,22.1,200,1010.80,38.6,1.5,0,...,14.8,104.2,21.9,1.7,2.7,0.2,Canada,55.200000,-119.40000,745.0
2,2014-10-11 12:00:00,2.9,2.3,95.0,1.9,250,1018.19,27.7,0.0,2,...,3.1,330.1,6.1,0.1,0.0,0.0,Canada,49.820000,-74.98000,381.1
3,2015-09-11 12:00:00,22.1,11.1,49.8,9.5,267,1015.79,23.9,0.0,4,...,11.4,282.9,20.7,3.8,6.3,0.7,Canada,54.280000,-111.45000,619.0
4,2016-06-22 12:00:00,17.0,10.0,63.6,35.2,260,1016.70,16.1,0.0,1,...,107.4,482.9,138.1,15.6,47.7,25.4,United States,37.618805,-122.37542,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183935,2012-09-19 12:00:00,22.5,15.4,63.0,35.2,180,1016.90,10.1,0.0,2,...,11.5,27.6,11.5,16.9,16.3,3.8,Canada,45.230000,-63.06000,23.5
1183936,2011-08-01 12:00:00,17.4,12.1,70.0,22.2,300,1018.30,49.7,0.0,10,...,20.6,334.2,35.6,6.1,12.8,2.5,Canada,49.350000,-124.16000,14.5
1183937,2017-07-09 12:00:00,21.0,15.0,68.7,13.0,200,1014.50,14.5,0.0,1,...,5.2,107.5,9.3,2.1,1.8,0.1,Canada,43.628613,-79.39500,76.8
1183938,2010-04-24 12:00:00,16.2,-1.4,31.0,25.9,220,1016.50,48.3,0.0,3,...,12.1,21.5,12.0,15.3,15.4,3.4,Canada,45.466667,-73.75000,36.0


In [3]:
train_data = data.iloc[:20000]       
train_data

,report_date,temperature,dew_point_temperature,relative_humidity,wind_speed,wind_direction,atmospheric_pressure,visibility,precipitation,rain_days,...,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,daily_severity_rating,country,latitude,longitude,elevation
0,2019-08-27 12:00:00,12.0,7.6,74.4,12.0,0,1015.13,30.0,2.60,0,...,6.9,51.1,10.3,0.9,0.6,0.0,United Kingdom,54.502224,-2.68500,252.0
1,2018-06-06 12:00:00,11.1,1.7,52.0,22.1,200,1010.80,38.6,1.50,0,...,14.8,104.2,21.9,1.7,2.7,0.2,Canada,55.200000,-119.40000,745.0
2,2014-10-11 12:00:00,2.9,2.3,95.0,1.9,250,1018.19,27.7,0.00,2,...,3.1,330.1,6.1,0.1,0.0,0.0,Canada,49.820000,-74.98000,381.1
3,2015-09-11 12:00:00,22.1,11.1,49.8,9.5,267,1015.79,23.9,0.00,4,...,11.4,282.9,20.7,3.8,6.3,0.7,Canada,54.280000,-111.45000,619.0
4,2016-06-22 12:00:00,17.0,10.0,63.6,35.2,260,1016.70,16.1,0.00,1,...,107.4,482.9,138.1,15.6,47.7,25.4,United States,37.618805,-122.37542,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2018-12-25 12:00:00,15.0,2.0,41.7,16.7,350,1016.50,16.1,22.10,0,...,1.1,19.1,1.9,0.4,0.1,0.0,United States,40.508972,-122.29339,153.9
19996,2015-11-28 12:00:00,-3.8,-12.0,54.2,6.4,241,1027.78,28.3,0.34,0,...,39.4,318.2,60.2,1.4,4.8,0.4,Canada,51.780000,-112.25000,845.0
19997,2018-09-14 12:00:00,4.5,-1.4,65.0,16.5,112,1024.23,23.2,0.00,5,...,71.0,748.1,114.7,4.2,18.4,4.7,Canada,50.300000,-112.86000,955.0
19998,2019-09-09 12:00:00,15.6,6.1,53.4,14.8,170,1022.70,24.0,0.00,2,...,3.6,80.0,6.4,2.7,2.0,0.1,Canada,50.116665,-91.90000,383.1


In [4]:
train_data = train_data.drop(['wind_direction','snow_on_ground','daily_severity_rating','latitude','longitude', 'elevation','visibility'], axis = 1)
train_data

,report_date,temperature,dew_point_temperature,relative_humidity,wind_speed,atmospheric_pressure,precipitation,rain_days,fine_fuel_moisture_code,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,country
0,2019-08-27 12:00:00,12.0,7.6,74.4,12.0,1015.13,2.60,0,64.1,6.9,51.1,10.3,0.9,0.6,United Kingdom
1,2018-06-06 12:00:00,11.1,1.7,52.0,22.1,1010.80,1.50,0,66.2,14.8,104.2,21.9,1.7,2.7,Canada
2,2014-10-11 12:00:00,2.9,2.3,95.0,1.9,1018.19,0.00,2,44.3,3.1,330.1,6.1,0.1,0.0,Canada
3,2015-09-11 12:00:00,22.1,11.1,49.8,9.5,1015.79,0.00,4,85.8,11.4,282.9,20.7,3.8,6.3,Canada
4,2016-06-22 12:00:00,17.0,10.0,63.6,35.2,1016.70,0.00,1,86.6,107.4,482.9,138.1,15.6,47.7,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2018-12-25 12:00:00,15.0,2.0,41.7,16.7,1016.50,22.10,0,49.9,1.1,19.1,1.9,0.4,0.1,United States
19996,2015-11-28 12:00:00,-3.8,-12.0,54.2,6.4,1027.78,0.34,0,78.9,39.4,318.2,60.2,1.4,4.8,Canada
19997,2018-09-14 12:00:00,4.5,-1.4,65.0,16.5,1024.23,0.00,5,84.0,71.0,748.1,114.7,4.2,18.4,Canada
19998,2019-09-09 12:00:00,15.6,6.1,53.4,14.8,1022.70,0.00,2,81.1,3.6,80.0,6.4,2.7,2.0,Canada


In [5]:
# Load saved PowerTransformer dictionary
with open(r'E:\Info_Srping\FWI_Prediction\models\Final_pkl_files\power_transformers.pkl', 'rb') as f:
    power_transformers = pickle.load(f)

# Load saved StandardScaler
with open(r'E:\Info_Srping\FWI_Prediction\models\Final_pkl_files\standard_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Load saved OneHotEncoding
with open(r'E:\Info_Srping\FWI_Prediction\models\Final_pkl_files\one_hot_encoder.pkl', 'rb') as f:
    ohe = pickle.load(f)


In [6]:
col = ['fine_fuel_moisture_code','duff_moisture_code', 'drought_code','build_up_index', 'initial_spread_index','fire_weather_index','wind_speed']
for col, pt in power_transformers.items():
    # Transform test column using fitted transformer
    transformed = pt.transform(train_data[col].values.reshape(-1, 1))
    train_data[col + '_transformed'] = transformed.flatten()


In [7]:
numeric_cols = train_data.select_dtypes(include=['float64', 'int']).columns
train_data[numeric_cols] = scaler.transform(train_data[numeric_cols])

In [8]:
train_data = train_data[['temperature', 'dew_point_temperature', 'relative_humidity',
        'atmospheric_pressure', 'precipitation', 'rain_days',
       'country', 'fine_fuel_moisture_code_transformed',
       'duff_moisture_code_transformed', 'drought_code_transformed',
       'build_up_index_transformed', 'initial_spread_index_transformed',
       'fire_weather_index_transformed', 'wind_speed_transformed']]

In [9]:
train_data['FFMC__TEMP'] = train_data['temperature'] * train_data['fine_fuel_moisture_code_transformed']
train_data['FFMC_RH'] = train_data['relative_humidity'] * train_data['fine_fuel_moisture_code_transformed']
train_data['RH_TEMP'] = train_data['temperature'] * train_data['relative_humidity']
train_data['RD_FFMC'] =  train_data['fine_fuel_moisture_code_transformed'] * train_data['rain_days']
train_data['DC_RD'] =  train_data['drought_code_transformed'] * train_data['rain_days']
train_data['WS_TENP'] =  train_data['wind_speed_transformed'] * train_data['fine_fuel_moisture_code_transformed']
train_data['IST_WS'] =  train_data['wind_speed_transformed'] * train_data['initial_spread_index_transformed']
train_data['ISI_FFMC'] =  train_data['initial_spread_index_transformed'] * train_data['fine_fuel_moisture_code_transformed']
train_data['ISI_RH'] =  train_data['initial_spread_index_transformed'] * train_data['relative_humidity']
train_data['DMC_TEMP'] =  train_data['duff_moisture_code_transformed'] * train_data['temperature']
train_data['DMC_RD'] =  train_data['duff_moisture_code_transformed'] * train_data['rain_days']
train_data['RD_TEMP'] =  train_data['rain_days'] * train_data['temperature']
train_data['DMC_DC'] =  train_data['drought_code_transformed'] * train_data['duff_moisture_code_transformed']
train_data['BUI_DC'] =  train_data['drought_code_transformed'] * train_data['build_up_index_transformed']
train_data['BUI_DMC'] =  train_data['duff_moisture_code_transformed'] * train_data['build_up_index_transformed']

C:\Users\DELL\AppData\Local\Temp\ipykernel_14708\1968526459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['FFMC__TEMP'] = train_data['temperature'] * train_data['fine_fuel_moisture_code_transformed']
C:\Users\DELL\AppData\Local\Temp\ipykernel_14708\1968526459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['FFMC_RH'] = train_data['relative_humidity'] * train_data['fine_fuel_moisture_code_transformed']
C:\Users\DELL\AppData\Local\Temp\ipykernel_14708\1968526459.py:3: SettingWi

In [10]:
encoded_array = ohe.transform(train_data[['country']])  
encoded_df = pd.DataFrame(encoded_array, columns=ohe.get_feature_names_out(['country']))
new_data_encoded = pd.concat([train_data.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


In [11]:
new_data_encoded = new_data_encoded.drop(['country'], axis =1)
new_data_encoded

,temperature,dew_point_temperature,relative_humidity,atmospheric_pressure,precipitation,rain_days,fine_fuel_moisture_code_transformed,duff_moisture_code_transformed,drought_code_transformed,build_up_index_transformed,...,DMC_RD,RD_TEMP,DMC_DC,BUI_DC,BUI_DMC,country_Canada,country_France,country_Spain,country_United Kingdom,country_United States
0,-0.594522,0.046346,0.762831,0.078602,0.011661,-0.542337,-0.803750,-0.581596,-0.969009,-0.611173,...,0.315421,0.322431,0.563571,0.592232,0.355456,0,0,0,1,0
1,-0.708704,-0.756624,-0.292205,-0.505897,-0.139434,-0.542337,-0.696235,-0.010055,-0.523163,-0.048529,...,0.005453,0.384356,0.005260,0.025389,0.000488,1,0,0,0,0
2,-1.749029,-0.674966,1.733088,0.491666,-0.345472,-0.109146,-1.558656,-1.095775,0.588132,-0.951194,...,0.119599,0.190899,-0.644460,-0.559428,1.042295,1,0,0,0,0
3,0.686853,0.522685,-0.395825,0.167695,-0.345472,0.324045,0.599011,-0.213236,0.403706,-0.093457,...,-0.069098,0.222572,-0.086085,-0.037729,0.019928,1,0,0,0,0
4,0.039822,0.372979,0.254153,0.290534,-0.345472,-0.325741,0.663822,1.742717,1.101788,1.678844,...,-0.567675,-0.012972,1.920105,1.849730,2.925750,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.213916,-0.715795,-0.777334,0.263536,2.690159,-0.542337,-1.389689,-1.594914,-1.386612,-1.545249,...,0.864981,0.116014,2.211527,2.142661,2.464538,0,0,0,0,1
19996,-2.599050,-2.621149,-0.188586,1.786204,-0.298770,-0.542337,0.080252,0.812881,0.543074,0.836654,...,-0.440855,1.409561,0.441454,0.454365,0.680100,1,0,0,0,0
19997,-1.546039,-1.178524,0.320093,1.306996,-0.345472,0.540641,0.456777,1.348900,1.813121,1.481110,...,0.729271,-0.835852,2.445720,2.685432,1.997870,1,0,0,0,0
19998,-0.137795,-0.157799,-0.226265,1.100464,-0.345472,-0.109146,0.237935,-1.007381,-0.705710,-0.921832,...,0.109951,0.015040,0.710919,0.650546,0.928636,1,0,0,0,0


In [13]:
new_data_encoded['BUI_SQRT'] = (new_data_encoded['build_up_index_transformed'].clip(lower=0)) ** 0.5
new_data_encoded['FFMC_SQRT'] = (new_data_encoded['fine_fuel_moisture_code_transformed'].clip(lower=0)) ** 0.5
new_data_encoded['ISI_SQRT'] = (new_data_encoded['initial_spread_index_transformed'].clip(lower=0)) ** 0.5

In [14]:
# Prepare features and target from your new test data
X = new_data_encoded.drop('fire_weather_index_transformed', axis = 1)
y = new_data_encoded['fire_weather_index_transformed']

In [15]:
def calculate_vif(X: pd.DataFrame):
    """
    Calculate VIF for each feature in the DataFrame.
    """
    # Add intercept for statsmodels
    X_with_const = add_constant(X)

    vif_data = pd.DataFrame()
    vif_data["Feature"] = X_with_const.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_with_const.values, i)
        for i in range(X_with_const.shape[1])
    ]

    # Drop the intercept row
    return vif_data.drop(index=0).reset_index(drop=True)

In [16]:
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                                Feature          VIF
0                           temperature    65.902893
1                 dew_point_temperature    61.262133
2                     relative_humidity    41.496939
3                  atmospheric_pressure     1.190577
4                         precipitation     1.845527
5                             rain_days     7.270217
6   fine_fuel_moisture_code_transformed    98.950654
7        duff_moisture_code_transformed  1098.948412
8              drought_code_transformed    46.117505
9            build_up_index_transformed  1524.806678
10     initial_spread_index_transformed   129.084399
11               wind_speed_transformed     9.159853
12                           FFMC__TEMP     4.748500
13                              FFMC_RH    16.782967
14                              RH_TEMP     3.601856
15                              RD_FFMC    13.177413
16                                DC_RD     7.703138
17                              WS_TENP     4.

In [18]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','build_up_index_transformed'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']

In [19]:
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                                Feature         VIF
0                           temperature   65.743509
1                 dew_point_temperature   61.126875
2                     relative_humidity   41.403430
3                  atmospheric_pressure    1.190538
4                         precipitation    1.845276
5                             rain_days    7.268450
6   fine_fuel_moisture_code_transformed   98.948448
7        duff_moisture_code_transformed   10.512012
8              drought_code_transformed    2.534670
9      initial_spread_index_transformed  129.060921
10               wind_speed_transformed    9.159610
11                           FFMC__TEMP    4.742318
12                              FFMC_RH   16.780175
13                              RH_TEMP    3.596453
14                              RD_FFMC   13.176000
15                                DC_RD    7.697990
16                              WS_TENP    4.995537
17                               IST_WS    7.529366
18          

In [20]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','build_up_index_transformed','country_United States','DMC_DC'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)

                                Feature         VIF
0                           temperature   65.629557
1                 dew_point_temperature   61.125559
2                     relative_humidity   41.378169
3                  atmospheric_pressure    1.190276
4                         precipitation    1.842507
5                             rain_days    7.261944
6   fine_fuel_moisture_code_transformed   98.899409
7        duff_moisture_code_transformed   10.466405
8              drought_code_transformed    2.533615
9      initial_spread_index_transformed  129.057612
10               wind_speed_transformed    9.156916
11                           FFMC__TEMP    4.734036
12                              FFMC_RH   16.776858
13                              RH_TEMP    3.595869
14                              RD_FFMC   13.169012
15                                DC_RD    7.681332
16                              WS_TENP    4.995498
17                               IST_WS    7.529333
18          

In [21]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','build_up_index_transformed','country_United States','DMC_DC','initial_spread_index_transformed'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)
  

                                Feature        VIF
0                           temperature  64.072744
1                 dew_point_temperature  60.024358
2                     relative_humidity  41.265406
3                  atmospheric_pressure   1.189556
4                         precipitation   1.841614
5                             rain_days   7.245276
6   fine_fuel_moisture_code_transformed  53.737636
7        duff_moisture_code_transformed  10.459706
8              drought_code_transformed   2.532312
9                wind_speed_transformed   2.824374
10                           FFMC__TEMP   4.721109
11                              FFMC_RH  16.306729
12                              RH_TEMP   3.592914
13                              RD_FFMC  12.894860
14                                DC_RD   7.681330
15                              WS_TENP   4.899904
16                               IST_WS   4.775878
17                             ISI_FFMC  18.968967
18                             

In [22]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','build_up_index_transformed',
                           'country_United States','DMC_DC','initial_spread_index_transformed',
                          'temperature'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)
  

                                Feature        VIF
0                 dew_point_temperature   1.823920
1                     relative_humidity   4.831310
2                  atmospheric_pressure   1.188576
3                         precipitation   1.841065
4                             rain_days   7.225096
5   fine_fuel_moisture_code_transformed  49.067961
6        duff_moisture_code_transformed  10.373074
7              drought_code_transformed   2.528382
8                wind_speed_transformed   2.822668
9                            FFMC__TEMP   4.702155
10                              FFMC_RH  15.637573
11                              RH_TEMP   3.339486
12                              RD_FFMC  12.834663
13                                DC_RD   7.677945
14                              WS_TENP   4.895071
15                               IST_WS   4.775660
16                             ISI_FFMC  18.656276
17                               ISI_RH  18.596091
18                             

In [23]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','build_up_index_transformed',
                           'country_United States','DMC_DC','initial_spread_index_transformed',
                          'temperature','fine_fuel_moisture_code_transformed'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)
  

                           Feature        VIF
0            dew_point_temperature   1.754641
1                relative_humidity   3.703044
2             atmospheric_pressure   1.188496
3                    precipitation   1.809475
4                        rain_days   6.984484
5   duff_moisture_code_transformed  10.149773
6         drought_code_transformed   2.528283
7           wind_speed_transformed   2.770807
8                       FFMC__TEMP   4.580747
9                          FFMC_RH  13.105334
10                         RH_TEMP   3.071783
11                         RD_FFMC  12.643654
12                           DC_RD   7.677766
13                         WS_TENP   4.880975
14                          IST_WS   4.748202
15                        ISI_FFMC   8.800242
16                          ISI_RH  12.836246
17                        DMC_TEMP   3.672261
18                          DMC_RD  14.964636
19                         RD_TEMP   2.385104
20                          BUI_DC

In [24]:
# Prepare features and target from your new test data
X = new_data_encoded.drop(['fire_weather_index_transformed','build_up_index_transformed',
                           'country_United States','DMC_DC','initial_spread_index_transformed',
                          'temperature','fine_fuel_moisture_code_transformed','DMC_RD'], axis = 1)
y = new_data_encoded['fire_weather_index_transformed']
# Example usage
vif_df = calculate_vif(X)
print(vif_df)
  

                           Feature        VIF
0            dew_point_temperature   1.743555
1                relative_humidity   3.687337
2             atmospheric_pressure   1.188379
3                    precipitation   1.809184
4                        rain_days   6.946056
5   duff_moisture_code_transformed  10.131029
6         drought_code_transformed   2.528262
7           wind_speed_transformed   2.761726
8                       FFMC__TEMP   4.565129
9                          FFMC_RH  13.103518
10                         RH_TEMP   3.071685
11                         RD_FFMC  10.473590
12                           DC_RD   3.048862
13                         WS_TENP   4.862692
14                          IST_WS   4.747639
15                        ISI_FFMC   8.662677
16                          ISI_RH  12.835261
17                        DMC_TEMP   3.615335
18                         RD_TEMP   2.311953
19                          BUI_DC   4.010227
20                         BUI_DMC